In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F

from DataLoader import CustomDataLoader
from UpNet import UpNet

In [2]:
class UpNet(nn.Module):
    def __init__(self):
        super(UpNet, self).__init__()
        # convolutional layer
        self.up_layer = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=1, padding=3)
        self.down_layer = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=7, stride=1, padding=3)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = F.elu(self.up_layer(x))
        x = F.elu(self.down_layer(x))        
        return x

In [3]:
model = UpNet()

In [4]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
path = ""
path_state_dict = path+"checkpoints/state_dict_01.pth"
path_model = path+"checkpoints/model_01.pth"
train_on_gpu = torch.cuda.is_available()


In [5]:
transform2Tensor = transforms.ToTensor()
transform2PIL = transforms.ToPILImage()
model.load_state_dict(torch.load(path_state_dict))
model.eval()

if train_on_gpu:
    model = model.cuda()

In [11]:
image_path = "res/this.png"

with torch.no_grad():
    image = Image.open(image_path)
    image_tensor = transform2Tensor(image)
    image_tensor = image_tensor.unsqueeze(0)

    if train_on_gpu:
        image_tensor = image_tensor.cuda()

    output = model(image_tensor)
    output = output.squeeze().cpu()


    image_trans = transform2PIL(output)
    image_trans.show()